In [29]:
import os
import numpy as np

files = ['measurement_m1.dat', 'measurement_m2.dat']
for file in files:
    with open(file,'r') as f:
        data = f.read()
        measurements = np.array(data.split('\n'))
        measurements = measurements[measurements != ''].astype(np.float32)
        print('{} media {} dp {}'.format(file, np.average(measurements), np.std(measurements)))

measurement_m1.dat media 0.08285761624574661 dp 0.006997148972004652
measurement_m2.dat media 0.1281004548072815 dp 0.008999202400445938
